<a href="https://colab.research.google.com/github/igjorque/IM_S6_DialogForAlert/blob/master/DAG_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Descarga e instalación

Para esta actividad, he escogido el siguiente dataset:

https://www.kaggle.com/deepak711/4-subject-data-text-classification

El objetivo con el mismo es el de clasificar asignaturas/temas. Sin embargo, del mismo sólo he mantenido tres: Computer_Science, History y Maths. El dataset cuenta además con datos para Biology, Geography y Physics.

A continuación, descargamos los datos de Google Drive y los descomprimimos:

In [ ]:
!gdown --id 1fE2IzrCoXLx4UvBERfZyi3VM2M_EOkZy

Downloading...
From: https://drive.google.com/uc?id=1fE2IzrCoXLx4UvBERfZyi3VM2M_EOkZy
To: /content/Data.zip
2.14MB [00:00, 68.6MB/s]


In [ ]:
!unzip Data

Archive:  Data.zip
   creating: Data/
   creating: Data/Computer_Science/
  inflating: Data/Computer_Science/Computer_Science100.txt  
  inflating: Data/Computer_Science/Computer_Science101.txt  
  inflating: Data/Computer_Science/Computer_Science102.txt  
  inflating: Data/Computer_Science/Computer_Science103.txt  
  inflating: Data/Computer_Science/Computer_Science104.txt  
  inflating: Data/Computer_Science/Computer_Science105.txt  
  inflating: Data/Computer_Science/Computer_Science106.txt  
  inflating: Data/Computer_Science/Computer_Science107.txt  
  inflating: Data/Computer_Science/Computer_Science108.txt  
  inflating: Data/Computer_Science/Computer_Science109.txt  
  inflating: Data/Computer_Science/Computer_Science110.txt  
  inflating: Data/Computer_Science/Computer_Science111.txt  
  inflating: Data/Computer_Science/Computer_Science112.txt  
  inflating: Data/Computer_Science/Computer_Science113.txt  
  inflating: Data/Computer_Science/Computer_Science114.txt  
  inflating

Para la realización de esta actividad, utilizamos FastAI. Instalamos el componente e importamos sus librerías:

In [ ]:
!pip install fastai --upgrade

     |████████████████████████████████| 204kB 7.6MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [ ]:
from fastai.text.all import *
import os
from google.colab import files
from sklearn.model_selection import train_test_split
import shutil

## Conjuntos de entrenamiento y test

El primer requisito para entrenar un modelo con FastAI es el de separar los datos en carpetas de train y test. Establecemos el path donde tendremos dichas carpetas:

In [ ]:
path = Path('Data')

Seguidamente, creamos las carpetas necesarias para los tres temas tanto en train como en test:

In [ ]:
(path/'train/Computer_Science').mkdir(parents=True,exist_ok=True)
(path/'train/History').mkdir(parents=True,exist_ok=True)
(path/'train/Maths').mkdir(parents=True,exist_ok=True)

(path/'test/Computer_Science').mkdir(parents=True,exist_ok=True)
(path/'test/History').mkdir(parents=True,exist_ok=True)
(path/'test/Maths').mkdir(parents=True,exist_ok=True)

Con train_test_split separamos los datos en conjuntos de entrenamiento y de test. A dicha función, le indicamos que utilice un tamaño de test del 20%:

In [ ]:
trainComputerScience, testComputerScience = train_test_split(get_text_files(path/'Computer_Science'),test_size=0.2,random_state=15)
trainHistory, testHistory = train_test_split(get_text_files(path/'History'),test_size=0.2,random_state=15)
trainMaths, testMaths = train_test_split(get_text_files(path/'Maths'),test_size=0.2,random_state=15)

La anterior función nos separa los identificadores (o las referencias) de los datos en conjuntos de entrenamiento y de test, pero ahora debemos moverlos a sus respectivas carpetas:

In [ ]:
for x in trainComputerScience:
  shutil.move(str(x),path/('train/Computer_Science/'+x.name))

for x in trainHistory:
  shutil.move(str(x),path/('train/History/'+x.name))

for x in trainMaths:
  shutil.move(str(x),path/('train/Maths/'+x.name))

for x in testComputerScience:
  shutil.move(str(x),path/('test/Computer_Science/'+x.name))

for x in testHistory:
  shutil.move(str(x),path/('test/History/'+x.name))

for x in testMaths:
  shutil.move(str(x),path/('test/Maths/'+x.name))

Los directorios antiguos ya no son necesarios, por lo que los eliminamos.

In [ ]:
shutil.rmtree('Data/Computer_Science')
shutil.rmtree('Data/History')
shutil.rmtree('Data/Maths')

## Modelo de lenguaje

Para crear el modelo de lenguaje, necesitamos realizar 4 pasos: "Tokenización", "Numericalización", "Organizar el dataset" y "Construir el modelo". Los tres primeros pasos no son necesarios, puesto que FastAI se encarga de realizarlos automáticamente en el cuarto paso. Aún así, veamos con detalle dicho proceso.

Empezamos por la Tokenización. Puesto que nos interesa tener un vocabulario lo más amplio posible, tomamos todos los ficheros de las carpetas train y test:

In [ ]:
files = get_text_files(path, folders = ['train', 'test'])

FastAI cuenta con su propio tokenizador, el cual cuenta con los dos siguientes pasos:

In [ ]:
wt = WordTokenizer()
tkn = Tokenizer(wt)

WordTokenizer() separa un texto en una lista de palabras, y Tokenizer() pasa todas las palabras a minúsculas, además de añadir ciertos tokens especiales empezados por 'xx'. Estos tokens indican ocurrencias en el texto: 'xxbos' indica el comienzo de un texto, 'xxunk' indica que la siguiente palabra es desconocida, etc.

El siguiente paso es la Numericalización, que consiste en asignar un identificador a cada token utilizando la clase Numericalize. Pero antes, debemos construir nuestro vocabulario, para lo que utilizaré las 300 primeras palabras del dataset:

In [ ]:
txts = L(o.open().read() for o in files[:2000])
toks300 = txts[:300].map(tkn)
num = Numericalize()
num.setup(toks300)

Para comprobar que se haya realizado correctamente, con la siguiente instrucción podemos ver las 20 primeras palabras del vocabulario:

In [ ]:
coll_repr(num.vocab,20)

"(#3576) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','\\n','the',',','.','to','of','a','and','in','is','-'...]"

El tercer paso es el de organizar el dataset en batches. Debemos obtener la X (secuencia de palabras numericalizadas de la primera a la penúltima) y la Y (secuencia de la segunda a la última) de nuestro dataset. Para ello, se utiliza la clase LMDataLoader:

In [ ]:
nums300 = toks300.map(num)
dl = LMDataLoader(nums300)

Dicho objeto que hemos construido parte el texto en bloques (o batches) de tamaño 64 por defecto, aunque más adelante veremos que podemos modificar ese número.

In [ ]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

La siguiente instrucción sirve simplemente para ver los bloques X e Y, y comprobar así que estan desplazados un token entre sí.

In [ ]:
print(' '.join(num.vocab[o] for o in x[0][:20]))
print(' '.join(num.vocab[o] for o in y[0][:20]))

xxbos 2.1 xxmaj introduction 48 
 2.2 xxmaj operations of the xxmaj computer xxmaj hardware 49 
 2.3 xxmaj operands
2.1 xxmaj introduction 48 
 2.2 xxmaj operations of the xxmaj computer xxmaj hardware 49 
 2.3 xxmaj operands of


Hasta este punto, todo lo anterior no sería necesario al trabajar con FastAI.

El último paso, sería el de Construir el modelo, el cual veremos a continuación.

## Construir el modelo

Con FastAI este sería el único paso que deberíamos realizar. Para comenzar, cargamos los datos (igual que realizamos en el primer paso del proceso) y, seguidamente, creamos el bloque de datos con la clase DataBlock.

In [ ]:
get_text = partial(get_text_files, folders=['train', 'test'])

dls_lm = DataBlock(blocks=TextBlock.from_folder(path, is_lm=True),
                  get_items=get_text, 
                  splitter=RandomSplitter(0.1)).dataloaders(path, path=path, bs=128, seq_len=80)

Podemos apreciar que en el objeto DataBlock hay una serie de parámetros que podemos ajustar, en concreto el 'bs' (o batch_size) que mencioné anteriormente.

Para comprobar que se ha realizado correctamente, podemos ver un par de elementos del batch:

In [ ]:
dls_lm.show_batch(max_n=2)

,text,text_
0,xxbos xxmaj glossary xxup xxunk \n global pointer xxmaj the register that is reserved ito instructions a dedicated instruction \n to point to static data . that is used to give a command to an 110 de \n guard xxmaj the first oftwo xxunk : ra bits kept on the vice and that specifies both the device num \n right during intermediate calculations of ber and the command word ( or xxunk \n floating - point numbers ; used to,xxmaj glossary xxup xxunk \n global pointer xxmaj the register that is reserved ito instructions a dedicated instruction \n to point to static data . that is used to give a command to an 110 de \n guard xxmaj the first oftwo xxunk : ra bits kept on the vice and that specifies both the device num \n right during intermediate calculations of ber and the command word ( or xxunk \n floating - point numbers ; used to improve
1,"he was active in the xxmaj xxunk xxmaj club , a branch \n violated by rejection of an abortion . xxmaj finally , xxmaj rehnquist of the xxmaj young xxmaj communists xxmaj league , and later joined the \n felt that the xxmaj court ruling in favor of legal abortion was xxmaj american xxmaj communist xxmaj party . xxmaj rosenberg was a civilian \n too sweeping of an act for a judicial body . employee of the xxup u.s .","was active in the xxmaj xxunk xxmaj club , a branch \n violated by rejection of an abortion . xxmaj finally , xxmaj rehnquist of the xxmaj young xxmaj communists xxmaj league , and later joined the \n felt that the xxmaj court ruling in favor of legal abortion was xxmaj american xxmaj communist xxmaj party . xxmaj rosenberg was a civilian \n too sweeping of an act for a judicial body . employee of the xxup u.s . xxmaj"


A continuación, construimos un objeto Learner que nos permita usar un modelo RNN.

In [ ]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy,Perplexity()]).to_fp16()

El siguiente paso es el de entrenar la red. Vamos a empezar entrenando sólo las últimas capas del modelo:

In [ ]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.270823,4.066672,0.295006,58.362427,00:22


Seguimos entrenándolo, pero esta vez descongelando la red y entrenándola al completo:

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.676642,3.903970,0.314135,49.598980,00:24
1,3.557436,3.866117,0.317379,47.756611,00:24
2,3.354585,3.746511,0.333846,42.373005,00:24
3,3.097387,3.716426,0.340840,41.117184,00:24
4,2.830359,3.760924,0.339902,42.988132,00:24
5,2.643018,3.941038,0.328443,51.471985,00:24
6,2.390057,3.898719,0.338314,49.339184,00:24
7,2.187838,3.958327,0.336543,52.369640,00:25
8,2.055951,4.000899,0.336129,54.647247,00:25
9,1.984059,4.019126,0.335129,55.652443,00:25


Obtenemos una precisión del 33% que, a pesar del alto sobreajuste de este modelo, es un resultado bastante bueno.

Para crear nuestro modelo de clasificación, no necesitamos la última capa de nuestro modelo. Con la siguiente instrucción, guardamos el modelo sin dicha capa. A esto se le conoce como "encoder".

In [ ]:
learn.save_encoder('finetuned')

Las dos siguientes instrucciones no son necesarias, sirven exclusivamente para guardar el modelo al completo en caso de querer reanudar la ejecución de este cuaderno desde este punto. La anterior instrucción ya guarda el modelo, salvo que sin su última capa.

In [ ]:
learn.export('model_subjects.pkl')

In [ ]:
Path().ls(file_exts='.pkl')

(#0) []

Para crear el clasificador, primero debemos cargar los datos.

In [ ]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

Para comprobar que se ha realizado correctamente, mostramos un batch del bloque anterior:

In [ ]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos 0 \t▁ literature \n xxmaj hanoi ; xxmaj norman xxmaj xxunk , author of xxmaj armies \t of \t the \t xxmaj night ; from xxmaj ireland , in 1995 ; and xxup j. xxup m. xxmaj xxunk , author of \n xxmaj james xxmaj xxunk ; xxmaj chaim xxmaj xxunk ; xxup j. xxup d. xxmaj xxunk , author of xxmaj the \t xxmaj life \t and \t xxmaj times \t of \t xxmaj michael \t xxup k , from xxmaj south xxmaj africa , \n xxmaj the \t xxmaj xxunk \t in \t the \t xxmaj xxunk ; xxmaj john xxmaj xxunk , author of xxmaj xxunk , \t▁ in 2003 . xxmaj prolific xxmaj south xxmaj african writer xxmaj xxunk xxmaj xxunk , \n xxmaj run and xxmaj the \t xxmaj xxunk \t of \t xxmaj xxunk ; xxmaj gore xxmaj xxunk , author author of",History
1,"xxbos xxmaj indonesian \t xxmaj communist \t xxmaj party \t ( pki ) \t▁ 05 \n xxmaj trail passing through xxmaj laos and xxmaj cambodia was the main xxmaj vietnam . xxmaj the xxmaj paris xxmaj peace xxmaj agreements on xxmaj vietnam were \n supply route for xxmaj north xxmaj vietnam to send xxunk xxunk signed on xxmaj january 27 , 1973 . xxmaj it was only a matter of \n ing supplies to the xxmaj vietcong in xxmaj south xxmaj vietnam . time before the communists would xxunk the final vic- \n xxmaj the xxup u.s . commitment to xxmaj south xxmaj vietnam strength- tory . xxmaj on xxmaj april 30 , 1975 , communist forces entered the \n ened during xxmaj president xxmaj john xxup f. xxmaj kennedy ’s administra- xxmaj south xxmaj vietnamese capital of xxmaj saigon . xxmaj the two xxmaj vietnams \n tion ( xxunk",History
2,"xxbos xxrep 3  \t▁ literature \n of xxmaj china ( prc ) . xxmaj lin continued to play a major role in as “ airport fiction , ” describing books that were sold \n both the government and the military and commanded to air xxunk with plenty of time to occupy . xxmaj digital \n “ volunteers ” from xxmaj china in the xxmaj korean xxmaj war ( 1950 – books in particular have allowed access to many old \n 53 ) ; he was promoted to the rank of marshal . and formerly out - of - print books and offer xxunk \n xxmaj in 1968 xxmaj mao embarked on the xxmaj great xxmaj xxunk xxunk functions giving readers and scholars the \n ian xxmaj cultural xxmaj revolution to attack his critics and ability to find information more quickly . xxmaj while this \n regain control of the",History


Construimos nuestro Learner (de igual forma que antes):

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

Ahora, para entrenarlo, cargamos el encoder guardado anteriormente:

In [ ]:
learn = learn.load_encoder('finetuned')

Ya podemos entrenar el clasificador, y comenzamos entrenando la última capa:

In [ ]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.930032,0.686385,0.753676,00:04


Podemos ver que la precisión es alta, pero que tenemos un alto subajuste.

Descongelamos dos capas, y volvemos a entrenar:

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.769494,0.377360,0.996324,00:05


La precisión sigue siendo alta, y el subajuste ahora es menor.

Descongelamos tres capas, y continuamos el entrenamiento:

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.650433,0.220279,1.000000,00:07


En este caso, hemos aumentado un poco la precisión, pero sigue habiendo el mismo subajuste que antes.

Para terminar, descongelamos todas las capas y entrenamos por última vez, en este caso:

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.588837,0.142127,1.000000,00:09
1,0.573432,0.099561,1.000000,00:09


Con esto, nuestro modelo ya estaría completamente entrenado y listo para funcionar. Podemos ver, además, que su precisión teórica es del 100%. Sin embargo, el modelo sigue siendo mejorable por la diferencia existente entre su train_loss y su valid_loss.